In [1]:
#https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html

import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [2]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [3]:
file = open('dataset_dsss.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
stegs = 'original ones zeros'.split()
for g in stegs:
    for filename in os.listdir(f'./stegoworksdsss/{g}'):
        songname = f'./stegoworksdsss/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rms = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset_dsss.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [4]:
data = pd.read_csv('dataset_dsss.csv')
data.head()
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(data.iloc[:, :-1])

print(X)
print(y)

   chroma_stft      rmse  spectral_centroid  spectral_bandwidth      rolloff  \
0     0.209805  0.053749         784.025099          785.480247  1298.761907   
1     0.203358  0.057514         779.086546          798.692074  1313.338145   
2     0.203358  0.057514         779.088291          798.697218  1313.338145   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3     mfcc4  ...  \
0            0.049807 -326.840668  195.610916 -17.375395  2.201211  ...   
1            0.049969 -317.415710  198.101364 -19.477287  6.534998  ...   
2            0.049969 -317.415649  198.101364 -19.477394  6.535070  ...   

     mfcc11    mfcc12    mfcc13    mfcc14    mfcc15    mfcc16    mfcc17  \
0 -6.887125 -6.038435 -3.639237 -1.142895 -1.839319 -3.073911 -4.248350   
1 -5.681645 -4.189331 -0.768128  2.027143  1.569866  0.305447 -1.788430   
2 -5.682170 -4.188948 -0.767991  2.026670  1.570264  0.305281 -1.788432   

     mfcc18    mfcc19    mfcc20  
0 -6.075161 -5.378838 -3.744005  
1 -4.8006

In [5]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=400,
                    batch_size=128)

Epoch 1/400
1/1 [==============================] - 0s 3ms/step - loss: 2.5150 - accuracy: 0.0000e+00
Epoch 2/400
1/1 [==============================] - 0s 3ms/step - loss: 2.1584 - accuracy: 0.5000
Epoch 3/400
1/1 [==============================] - 0s 9ms/step - loss: 1.9034 - accuracy: 0.5000
Epoch 4/400
1/1 [==============================] - 0s 2ms/step - loss: 1.6852 - accuracy: 1.0000
Epoch 5/400
1/1 [==============================] - 0s 4ms/step - loss: 1.4979 - accuracy: 1.0000
Epoch 6/400
1/1 [==============================] - 0s 2ms/step - loss: 1.3220 - accuracy: 1.0000
Epoch 7/400
1/1 [==============================] - 0s 4ms/step - loss: 1.1594 - accuracy: 1.0000
Epoch 8/400
1/1 [==============================] - 0s 3ms/step - loss: 1.0100 - accuracy: 1.0000
Epoch 9/400
1/1 [==============================] - 0s 3ms/step - loss: 0.8746 - accuracy: 1.0000
Epoch 10/400
1/1 [==============================] - 0s 2ms/step - loss: 0.7515 - accuracy: 1.0000
Epoch 11/400
1/1 [=======

1/1 [==============================] - 0s 997us/step - loss: 5.2093e-05 - accuracy: 1.0000
Epoch 83/400
1/1 [==============================] - 0s 1ms/step - loss: 5.1557e-05 - accuracy: 1.0000
Epoch 84/400
1/1 [==============================] - 0s 1ms/step - loss: 5.1080e-05 - accuracy: 1.0000
Epoch 85/400
1/1 [==============================] - 0s 3ms/step - loss: 5.0663e-05 - accuracy: 1.0000
Epoch 86/400
1/1 [==============================] - 0s 2ms/step - loss: 5.0186e-05 - accuracy: 1.0000
Epoch 87/400
1/1 [==============================] - 0s 3ms/step - loss: 4.9769e-05 - accuracy: 1.0000
Epoch 88/400
1/1 [==============================] - 0s 2ms/step - loss: 4.9292e-05 - accuracy: 1.0000
Epoch 89/400
1/1 [==============================] - 0s 2ms/step - loss: 4.8815e-05 - accuracy: 1.0000
Epoch 90/400
1/1 [==============================] - 0s 4ms/step - loss: 4.8517e-05 - accuracy: 1.0000
Epoch 91/400
1/1 [==============================] - 0s 3ms/step - loss: 4.8159e-05 - accuracy

1/1 [==============================] - 0s 1ms/step - loss: 2.7239e-05 - accuracy: 1.0000
Epoch 162/400
1/1 [==============================] - 0s 4ms/step - loss: 2.7001e-05 - accuracy: 1.0000
Epoch 163/400
1/1 [==============================] - 0s 3ms/step - loss: 2.6881e-05 - accuracy: 1.0000
Epoch 164/400
1/1 [==============================] - 0s 2ms/step - loss: 2.6703e-05 - accuracy: 1.0000
Epoch 165/400
1/1 [==============================] - 0s 2ms/step - loss: 2.6405e-05 - accuracy: 1.0000
Epoch 166/400
1/1 [==============================] - 0s 1ms/step - loss: 2.6226e-05 - accuracy: 1.0000
Epoch 167/400
1/1 [==============================] - 0s 2ms/step - loss: 2.5987e-05 - accuracy: 1.0000
Epoch 168/400
1/1 [==============================] - 0s 12ms/step - loss: 2.5808e-05 - accuracy: 1.0000
Epoch 169/400
1/1 [==============================] - 0s 5ms/step - loss: 2.5630e-05 - accuracy: 1.0000
Epoch 170/400
1/1 [==============================] - ETA: 0s - loss: 2.5332e-05 - accu

1/1 [==============================] - 0s 1ms/step - loss: 1.5557e-05 - accuracy: 1.0000
Epoch 240/400
1/1 [==============================] - 0s 6ms/step - loss: 1.5378e-05 - accuracy: 1.0000
Epoch 241/400
1/1 [==============================] - 0s 3ms/step - loss: 1.5318e-05 - accuracy: 1.0000
Epoch 242/400
1/1 [==============================] - 0s 1ms/step - loss: 1.5139e-05 - accuracy: 1.0000
Epoch 243/400
1/1 [==============================] - 0s 6ms/step - loss: 1.5080e-05 - accuracy: 1.0000
Epoch 244/400
1/1 [==============================] - 0s 2ms/step - loss: 1.4961e-05 - accuracy: 1.0000
Epoch 245/400
1/1 [==============================] - 0s 3ms/step - loss: 1.4901e-05 - accuracy: 1.0000
Epoch 246/400
1/1 [==============================] - 0s 2ms/step - loss: 1.4841e-05 - accuracy: 1.0000
Epoch 247/400
1/1 [==============================] - 0s 2ms/step - loss: 1.4722e-05 - accuracy: 1.0000
Epoch 248/400
1/1 [==============================] - 0s 2ms/step - loss: 1.4603e-05 - a

1/1 [==============================] - 0s 3ms/step - loss: 1.0252e-05 - accuracy: 1.0000
Epoch 319/400
1/1 [==============================] - 0s 5ms/step - loss: 1.0192e-05 - accuracy: 1.0000
Epoch 320/400
1/1 [==============================] - 0s 2ms/step - loss: 1.0133e-05 - accuracy: 1.0000
Epoch 321/400
1/1 [==============================] - 0s 5ms/step - loss: 1.0133e-05 - accuracy: 1.0000
Epoch 322/400
1/1 [==============================] - 0s 3ms/step - loss: 1.0133e-05 - accuracy: 1.0000
Epoch 323/400
1/1 [==============================] - 0s 3ms/step - loss: 1.0014e-05 - accuracy: 1.0000
Epoch 324/400
1/1 [==============================] - 0s 6ms/step - loss: 1.0014e-05 - accuracy: 1.0000
Epoch 325/400
1/1 [==============================] - 0s 7ms/step - loss: 1.0014e-05 - accuracy: 1.0000
Epoch 326/400
1/1 [==============================] - 0s 994us/step - loss: 9.9539e-06 - accuracy: 1.0000
Epoch 327/400
1/1 [==============================] - 0s 2ms/step - loss: 9.8943e-06 -

1/1 [==============================] - 0s 5ms/step - loss: 7.4505e-06 - accuracy: 1.0000
Epoch 398/400
1/1 [==============================] - 0s 4ms/step - loss: 7.4505e-06 - accuracy: 1.0000
Epoch 399/400
1/1 [==============================] - 0s 1ms/step - loss: 7.4505e-06 - accuracy: 1.0000
Epoch 400/400
1/1 [==============================] - 0s 1ms/step - loss: 7.4505e-06 - accuracy: 1.0000


In [7]:
data.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.209805,0.053749,784.025099,785.480247,1298.761907,0.049807,-326.840668,195.610916,-17.375395,2.201211,...,-6.038435,-3.639237,-1.142895,-1.839319,-3.073911,-4.248350,-6.075161,-5.378838,-3.744005,original
1,0.203358,0.057514,779.086546,798.692074,1313.338145,0.049969,-317.415710,198.101364,-19.477287,6.534998,...,-4.189331,-0.768128,2.027143,1.569866,0.305447,-1.788430,-4.800602,-4.031153,-2.009180,ones
2,0.203358,0.057514,779.088291,798.697218,1313.338145,0.049969,-317.415649,198.101364,-19.477394,6.535070,...,-4.188948,-0.767991,2.026670,1.570264,0.305281,-1.788432,-4.800697,-4.031086,-2.008828,zeros
